In [ ]:
import pandas as pd
import torch
import gc
torch.cuda.empty_cache()
gc.collect()
checkpoints = [
    "facebook/wav2vec2-large-960h-lv60-self",
    "classla/wav2vec2-large-slavic-parlaspeech-hr",
    "facebook/wav2vec2-large-slavic-voxpopuli-v2",
]
from utils import train_model, eval_model
results = []
for checkpoint in checkpoints:
    train_config = dict(
        model_name_or_path = checkpoint,
        TASK = "emotion_15epochs",
        NUM_EPOCH = 15,
        output_column = "target",
        input_column = "path",
        data_files = {
            "train": "001_train.csv",
            "validation": "001_dev.csv",
        },
    # clip_seconds = -1
    )
    torch.cuda.empty_cache()
    gc.collect()
    output_dir = train_model(train_config)
    torch.cuda.empty_cache()
    gc.collect()
    import numpy as np
    for split in "dev test".split():
        eval_config = dict(
            output_column = "target",
            model_name_or_path= output_dir + "/checkpoint-2640",
            eval_file= f"001_{split}.csv"
        )
        torch.cuda.empty_cache()
        gc.collect()
        y_true, y_pred = eval_model(eval_config)
        results.append({**eval_config, 
                        **train_config,
                        "split": split,
                        "y_true": y_true, 
                        "y_pred": y_pred})
        content = pd.DataFrame(data=results).to_json(None, 
                                           orient="records",
                                           lines=True,
                                           )
        with open("003_results.jsonl", "a") as f:
            f.writelines(content)

In [ ]:
import pandas as pd

checkpoints = [
    # "facebook/wav2vec2-large-960h-lv60-self",
    "classla/wav2vec2-large-slavic-parlaspeech-hr",
    # "facebook/wav2vec2-large-slavic-voxpopuli-v2",
]
from utils import train_model, eval_model
results = []

for checkpoint in checkpoints:
    train_config = dict(
        model_name_or_path = checkpoint,
        TASK = "emotion_15epochs",
        NUM_EPOCH = 15,
        output_column = "target",
        input_column = "path",
        data_files = {
            "train": "001_train.csv",
            "validation": "001_dev.csv",
        },
    clip_seconds = 10 if checkpoint == "classla/wav2vec2-large-slavic-parlaspeech-hr" else -1
    )
    output_dir = train_model(train_config)
    for split in "dev test".split():
        eval_config = dict(
            output_column = "target",
            model_name_or_path= output_dir + "/checkpoint-2640",
            eval_file= f"001_{split}.csv"
        )
        y_true, y_pred = eval_model(eval_config)
        results.append({**eval_config, 
                        **train_config,
                        "split": split,
                        "y_true": y_true, 
                        "y_pred": y_pred})
        content = pd.DataFrame(data=results).to_json(None, 
                                           orient="records",
                                           lines=True,
                                           )
        with open("003_results.jsonl", "a") as f:
            f.writelines(content)